#Gắn Google Drive:
Chạy cell này để kết nối notebook với Google Drive của bạn

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Cài đặt các Thư viện cần thiết:

In [ ]:
!pip install -q transformers datasets accelerate peft huggingface_hub
!pip install -q bitsandbytes
!pip install -q pandas
!pip install PyPDF2

In [6]:
import google.generativeai as genai
from datasets import Dataset
import ast # Dùng để chuyển đổi chuỗi thành dictionary một cách an toàn
import pandas as pd
import json
import time
from tqdm import tqdm
import os
from google.colab import userdata
from google.api_core import exceptions
import PyPDF2
import torch
import re
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)

Đăng nhập Hugging Face

In [ ]:
# Login to Hugging Face
from huggingface_hub import login

try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("Successfully logged in to Hugging Face.")
except Exception as e:
    print(f"Could not log in. Make sure you have set the HF_TOKEN secret. Error: {e}")

#Chuẩn bị Dữ liệu

In [3]:
# Define paths
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

GDRIVE_PATH = '/content/drive/My Drive/CEH_project/'
PDF_FOLDER_PATH = os.path.join(GDRIVE_PATH, 'data/Bo_A')
PROCESSED_DATA_PATH = os.path.join(GDRIVE_PATH, 'processed_data')

B1_CSV_PATH = os.path.join('/content/drive/MyDrive/CEH_project/data/Bo_B1/B1.csv')
OUTPUT_CSV_PATH = os.path.join(PROCESSED_DATA_PATH, 'generated_questions_for_llama3.csv')
OUTPUT_CSV_PATH2 = os.path.join(PROCESSED_DATA_PATH, 'generated_questions_for_llama3_reasoning.csv')
RESULTS_CSV_PATH_BASE = os.path.join(GDRIVE_PATH, 'evaluation_results_B1_BASE_MODEL.csv')

FINAL_ADAPTER_PATH = os.path.join(GDRIVE_PATH, "final_checkpoint")

In [ ]:
def extract_text_from_pdfs(folder_path):
    """Trích xuất văn bản từ tất cả các file PDF trong một thư mục."""
    full_text = ""
    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]
    print(f"Tìm thấy {len(pdf_files)} file PDF trong thư mục '{folder_path}'.")

    for filename in pdf_files:
        path = os.path.join(folder_path, filename)
        print(f"Đang đọc file: {filename}...")
        try:
            with open(path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    full_text += page.extract_text() + "\n"
        except Exception as e:
            print(f"Lỗi khi đọc file {filename}: {e}")
    return full_text

In [ ]:
def chunk_text(text, chunk_size, chunk_overlap):
    """Chia văn bản thành các đoạn nhỏ hơn có sự chồng lấn."""
    if not text:
        return []
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap
    return chunks

In [ ]:
def generate_mcq_from_chunk(chunk, max_retries=5):
    """
    Sử dụng Gemini để tạo một câu hỏi trắc nghiệm từ một đoạn văn bản.
    Output sẽ ở định dạng JSON.
    """
    model = genai.GenerativeModel('gemini-2.5-flash-lite')
    prompt = f"""
You are an expert Certified Ethical Hacker (CEH) exam question creator. Your task is to create a single, high-quality, **scenario-based** multiple-choice question based ONLY on the provided context.

**Scenario Rules:**
1.  **Create a Narrative:** Invent a short story or scenario involving a persona (e.g., 'an attacker named Sarah,' 'a security analyst named Alex,' 'an employee named David').
2.  **Apply the Context:** The story must demonstrate the practical application of a key concept, tool, or technique described in the `Context` block below.
3.  **Test the Application:** The question must ask the user to identify the tool, technique, concept, or outcome *within the context of your created scenario*.
4.  **Provide the Reasoning:** This is the most critical part. Explain in detail WHY the answer is correct.

**General Rules:**
1.  The question must be solvable using ONLY the information from the provided `Context`.
2.  **IMPORTANT:** The question must be phrased directly. It **MUST NOT** start with phrases like "According to the text...", "Based on the passage...", or any similar phrase that refers to the source text.
3.  There must be exactly four options (A, B, C, D), with one correct answer and three plausible distractors.
4.  Your entire output MUST be a single, valid JSON object. DO NOT add any explanatory text, markdown formatting like ```json, or any words before or after the JSON object.
5.  The JSON object must strictly adhere to the following structure:
{{
  "question": "Your generated question here.",
  "options": {{
    "A": "Option A.",
    "B": "Option B.",
    "C": "Option C.",
    "D": "Option D."
  }},
  "answer_key": "The letter of the correct option (e.g., 'C').",
  "reasoning": "The detailed explanation for why the answer is correct"
}}
**Context:**
---
{chunk}
---

**Your JSON Output:**
"""

    delay = 2  # Thời gian chờ ban đầu là 2 giây
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            cleaned_response = response.text.strip().replace('```json', '').replace('```', '')
            data = json.loads(cleaned_response)

            if "question" in data and "options" in data and "answer_key" in data:
                return data  # Thành công, trả về kết quả
            else:
                print("Lỗi: JSON nhận được không đúng cấu trúc.")
                return None

        # Bắt lỗi 429 (ResourceExhausted) một cách cụ thể
        except exceptions.ResourceExhausted as e:
            print(f"Lỗi 429: Vượt quá giới hạn truy vấn. Đang thử lại sau {delay} giây... (Lần thử {attempt + 1}/{max_retries})")
            time.sleep(delay)
            delay *= 2  # Gấp đôi thời gian chờ cho lần thử tiếp theo (Exponential Backoff)

        except json.JSONDecodeError:
            print(f"Lỗi giải mã JSON. Phản hồi từ API có thể không hợp lệ. (Lần thử {attempt + 1}/{max_retries})")
            time.sleep(delay) # Vẫn chờ trước khi thử lại
            delay *= 2

        except Exception as e:
            print(f"Một lỗi không xác định đã xảy ra: {e}. Đang thử lại sau {delay} giây...")
            time.sleep(delay)
            delay *= 2

    print(f"Không thể tạo câu hỏi cho đoạn văn bản này sau {max_retries} lần thử.")
    return None

In [ ]:
# Cấu hình phân đoạn văn bản
CHUNK_SIZE = 5000 # Số ký tự trong mỗi đoạn
CHUNK_OVERLAP = 500 # Số ký tự chồng lấn giữa các đoạn
# --------------------------------------------------------------------------
text_chunks = None
if not GOOGLE_API_KEY or GOOGLE_API_KEY == "YOUR_GOOGLE_API_KEY":
    print("Vui lòng thiết lập GOOGLE_API_KEY của bạn trong code.")
else:
    # 1. Trích xuất văn bản từ tất cả các file PDF
    raw_text = extract_text_from_pdfs(PDF_FOLDER_PATH)

    # 2. Phân đoạn văn bản
    text_chunks = chunk_text(raw_text, CHUNK_SIZE, CHUNK_OVERLAP)
    print(f"Đã chia văn bản thành {len(text_chunks)} đoạn.")

Tìm thấy 3 file PDF trong thư mục '/content/drive/My Drive/CEH_project/data/Bo_A'.
Đang đọc file: CEH_Certified_Ethical_Hacker_Bundle,_5th_Edition_Matt_Walker_2022-1.pdf...
Đang đọc file: Sybex_CEH_v10_Certified_Ethical.pdf...
Đang đọc file: CEH v10_ EC-Council Certified E - IP Specialist-1.pdf...
Đã chia văn bản thành 953 đoạn.


In [ ]:
text_chunks[0]

's collected publicly, mostly\nfrom social media or other sources.\nPayload\nThe payload  referrs to the actual section of information or data in a frame as\nopposed to automatically generated metadata. In information security,\nPayload is a section or part of a malicious and exploited code that causes the\npotentially harmful activity and actions such as exploit, opening backdoors,\nand hijacking.\nBot\nThe bots are software that is used to control the target remotely and to\nexecute predefined tasks. It is capable to run automated scripts over the\ninternet. The bots are also known as for Internet Bot or Web Robot.  These\nBots can be used for Social purposes such as Chatterbots, Commercial\npurpose or intended Malicious Purpose such as Spambots, Viruses, and\nWorms spreading, Botnets, DDoS attacks.\nElements of Information Security\nConfidentiality\nWe want to make sure that our secret and sensitive data is secure.\nConfidentiality means that only authorized persons can work with an

In [ ]:
# Thiết lập API Key
genai.configure(api_key=GOOGLE_API_KEY)

# 3. Tạo câu hỏi từ mỗi đoạn
all_questions = []
for chunk in tqdm(text_chunks, desc="Đang tạo câu hỏi"):
    mcq_data = generate_mcq_from_chunk(chunk)
    if mcq_data:
        all_questions.append({
            'question': mcq_data['question'],
            'options': mcq_data['options'],
            'ground_truth': mcq_data['answer_key'],
            'reasoning': mcq_data['reasoning']
        })
    time.sleep(4)

print(f"Tạo thành công {len(all_questions)} câu hỏi.")

In [ ]:
all_questions

[{'question': "An ethical hacker, Sarah, is tasked with assessing the security posture of a web application. She suspects that sensitive information might be exposed through directory listings. She uses a tool that systematically crawls the website's structure, looking for publicly accessible directories and files that are not explicitly linked from the main pages. This process helps her identify potential vulnerabilities such as exposed configuration files or backup archives. What is Sarah most likely performing?",
  'options': {'A': 'Vulnerability Scanning',
   'B': 'Web Application Penetration Testing',
   'C': 'Directory Brute-Forcing',
   'D': 'Social Engineering'},
  'ground_truth': 'C',
  'reasoning': "The scenario describes Sarah using a tool to systematically crawl a website's structure and find publicly accessible directories and files that are not linked from main pages. This is the core definition of directory brute-forcing, a technique used to discover hidden or unlinked d

Lưu câu hỏi vào file

In [ ]:
if all_questions:
        df = pd.DataFrame(all_questions)

        # Kiểm tra xem file CSV đã tồn tại hay chưa
        file_exists = os.path.exists(OUTPUT_CSV_PATH)

        print(f"Đang ghi {len(df)} câu hỏi mới vào file '{OUTPUT_CSV_PATH}'...")

        # Sử dụng mode='a' để ghi tiếp và chỉ ghi header nếu file chưa tồn tại
        df.to_csv(
            OUTPUT_CSV_PATH2,
            mode='a',
            header=not file_exists,
            index=True,
            index_label='index' # Đảm bảo tên cột index luôn là 'index'
        )

        if file_exists:
            print("Hoàn tất. Đã ghi tiếp vào file đã có.")
        else:
            print("Hoàn tất. Đã tạo file mới và ghi dữ liệu.")
else:
        print("Không có câu hỏi nào được tạo để ghi.")

Đang ghi 890 câu hỏi mới vào file '/content/drive/My Drive/CEH_project/processed_data/generated_questions_for_llama3.csv'...
Hoàn tất. Đã ghi tiếp vào file đã có.


Tạo Dữ liệu Huấn luyện

In [ ]:
# Hàm để tạo prompt huấn luyện
def create_training_prompt(row):
    try:
        # Chuyển đổi chuỗi options thành dictionary
        options_dict = ast.literal_eval(row['options'])
        # Định dạng lại các options
        formatted_options = "\n".join([f"{key}. {value}" for key, value in options_dict.items()])

        # Tạo prompt cuối cùng
        prompt = f"""### Instruction:
Analyze the following multiple-choice question from a Certified Ethical Hacker (CEH) context. Start your response by stating the single letter of the correct option (e.g., A, B, C, or D). Then, provide a detailed explanation justifying your choice and explaining why the other options are incorrect.

### Question:
{row['question']}
{formatted_options}

### Answer:
{row['answer_key']}. {row['reasoning']}
        """
        return {"text": prompt}
    except Exception as e:
        return None

In [ ]:
df = pd.read_csv(OUTPUT_CSV_PATH2)
print(f"Đã tải {len(df)} câu hỏi từ dữ liệu CSV.")

# Tạo cột 'text' mới
df['text'] = df.apply(create_training_prompt, axis=1)
df = df.dropna(subset=['text']) # Bỏ các dòng lỗi
df['text'] = df['text'].apply(lambda x: x['text'])

# Chuyển đổi pandas DataFrame thành Hugging Face Dataset
train_dataset = Dataset.from_pandas(df[['text']])
processed_file_path = os.path.join(PROCESSED_DATA_PATH, "finetuning_prompts.jsonl") # Dùng .jsonl cho rõ ràng

with open(processed_file_path, 'w', encoding='utf-8') as f:
    for record in train_dataset:
        # Chuyển mỗi record (dictionary) thành một chuỗi JSON
        json_record = json.dumps(record, ensure_ascii=False)
        # Ghi chuỗi JSON đó và thêm một dòng mới
        f.write(json_record + '\n')

print(f"\nĐã xử lý và lưu dữ liệu vào file: {processed_file_path}")
print("\nVí dụ về một prompt huấn luyện:")
print("-----------------------------------------")
print(train_dataset[0]['text'])
print("-----------------------------------------")

Đã tải 890 câu hỏi từ dữ liệu CSV.

Đã xử lý và lưu dữ liệu vào file: /content/drive/My Drive/CEH_project/processed_data/finetuning_prompts.jsonl

Ví dụ về một prompt huấn luyện:
-----------------------------------------
### Instruction:
Analyze the following multiple-choice question from a Certified Ethical Hacker (CEH) context. Start your response by stating the single letter of the correct option (e.g., A, B, C, or D). Then, provide a detailed explanation justifying your choice and explaining why the other options are incorrect.

### Question:
An ethical hacker, Sarah, is tasked with assessing the security posture of a web application. She suspects that sensitive information might be exposed through directory listings. She uses a tool that systematically crawls the website's structure, looking for publicly accessible directories and files that are not explicitly linked from the main pages. This process helps her identify potential vulnerabilities such as exposed configuration files 

#Fine-tuning Mô hình bằng QLoRA

In [ ]:
# Định nghĩa Model ID và đường dẫn Output ---
MODEL_ID = "meta-llama/Llama-3.2-1B"
print(f"Model sẽ được lưu tại: {GDRIVE_PATH}")

Model sẽ được lưu tại: /content/drive/My Drive/CEH_project/


In [ ]:
# Tải Model và Tokenizer (với lượng tử hóa 4-bit) ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

#3. Cấu hình LoRA ---
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("Cấu hình LoRA hoàn tất. Các tham số có thể huấn luyện:")
model.print_trainable_parameters()


# Tokenize bộ dữ liệu trước khi huấn luyện

def tokenize_function(examples):
    return tokenizer(examples["text"])

# Áp dụng hàm tokenize cho toàn bộ dataset
# batched=True giúp xử lý nhanh hơn
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Sau bước này, `tokenized_dataset` sẽ có các cột: input_ids, attention_mask
print("\nVí dụ về một mẫu dữ liệu đã được tokenize:")
print(tokenized_dataset[0])


# Thiết lập Training Arguments ---
training_args = TrainingArguments(
    output_dir=GDRIVE_PATH,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_strategy="epoch",
    optim="paged_adamw_8bit",
)


# Khởi tạo và Bắt đầu Huấn luyện ---
# Sử dụng DataCollatorForLanguageModeling.
# Nó sẽ tự động tạo `labels` từ `input_ids`, đây là cách chuẩn cho việc huấn luyện mô hình ngôn ngữ.
# mlm=False có nghĩa là chúng ta đang làm Causal Language Modeling (dự đoán từ tiếp theo), không phải Masked Language Modeling (như BERT).
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # Sử dụng dataset đã được tokenize
    tokenizer=tokenizer,
    data_collator=data_collator,      # Sử dụng data collator chuẩn
)

print("\nBắt đầu quá trình fine-tuning...")
trainer.train()

# Lưu adapter cuối cùng ---
trainer.save_model(FINAL_ADAPTER_PATH)
print(f"\nQuá trình fine-tuning hoàn tất. Adapter đã được lưu tại: {FINAL_ADAPTER_PATH}")

Cấu hình LoRA hoàn tất. Các tham số có thể huấn luyện:
trainable params: 6,815,744 || all params: 1,242,630,144 || trainable%: 0.5485


Map:   0%|          | 0/890 [00:00<?, ? examples/s]

/tmp/ipython-input-6-4286216806.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



Ví dụ về một mẫu dữ liệu đã được tokenize:
{'input_ids': [128000, 14711, 30151, 512, 2127, 56956, 279, 2768, 5361, 63726, 3488, 505, 264, 36542, 14693, 950, 89165, 320, 2152, 39, 8, 2317, 13, 5256, 701, 2077, 555, 28898, 279, 3254, 6661, 315, 279, 4495, 3072, 320, 68, 1326, 2637, 362, 11, 426, 11, 356, 11, 477, 423, 570, 5112, 11, 3493, 264, 11944, 16540, 1120, 7922, 701, 5873, 323, 26073, 3249, 279, 1023, 2671, 527, 15465, 382, 14711, 16225, 512, 2127, 31308, 55222, 11, 21077, 11, 374, 51920, 449, 47614, 279, 4868, 48378, 315, 264, 3566, 3851, 13, 3005, 30861, 430, 16614, 2038, 2643, 387, 15246, 1555, 6352, 26544, 13, 3005, 5829, 264, 5507, 430, 60826, 24877, 4835, 279, 3997, 596, 6070, 11, 3411, 369, 17880, 15987, 29725, 323, 3626, 430, 527, 539, 21650, 10815, 505, 279, 1925, 6959, 13, 1115, 1920, 8779, 1077, 10765, 4754, 52227, 1778, 439, 15246, 6683, 3626, 477, 16101, 39104, 13, 3639, 374, 21077, 1455, 4461, 16785, 5380, 32, 13, 81211, 2968, 2522, 6073, 198, 33, 13, 5000, 7473, 13

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: damdang319 (damdang319-vietnam-national-university-hanoi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,1.737900
40,1.455300
60,1.410600
80,1.394100
100,1.366900
120,1.349300
140,1.340900
160,1.363000
180,1.329400
200,1.311300


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)



Quá trình fine-tuning hoàn tất. Adapter đã được lưu tại: /content/drive/My Drive/CEH_project/final_checkpoint


#Test trên B1.csv

Test trên mô hình đã finetune

In [ ]:
# Tải mô hình đã fine-tune để Inference ---
# Sử dụng lượng tử hóa 4-bit để tải mô hình lên GPU của Colab
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

print(f"Đang tải kiến trúc mô hình gốc: {MODEL_ID}")
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

print(f"Đang tải adapter đã fine-tune từ: {FINAL_ADAPTER_PATH}")
# Tải adapter LoRA và hợp nhất nó vào mô hình gốc để tăng tốc độ inference
try:
    model = PeftModel.from_pretrained(base_model, FINAL_ADAPTER_PATH)
    model = model.merge_and_unload()
    model.eval() # Chuyển mô hình sang chế độ đánh giá
    print("Tải và hợp nhất mô hình thành công.")
except Exception as e:
    print(f"LỖI: Không thể tải adapter từ {FINAL_ADAPTER_PATH}. Hãy chắc chắn đường dẫn chính xác. Lỗi: {e}")
    raise


# Tải và chuẩn bị Dữ liệu Test (B1.csv) ---
try:
    test_df = pd.read_csv(B1_CSV_PATH)
    # Xử lý các dòng có thể bị lỗi
    test_df.dropna(subset=['question', 'ground_truth'], inplace=True)
    print(f"Đã tải {len(test_df)} câu hỏi từ {B1_CSV_PATH} để đánh giá.")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file B1.csv tại {B1_CSV_PATH}. Hãy chắc chắn bạn đã tải file lên đúng thư mục trên Google Drive.")
    raise


# Vòng lặp Đánh giá ---
correct_answers = 0
results_list = []

# (QUAN TRỌNG) Prompt template này PHẢI GIỐNG HỆT với prompt bạn đã dùng để fine-tune
prompt_template = """### Instruction:
Analyze the following multiple-choice question from a Certified Ethical Hacker (CEH) context. Start your response by stating the single letter of the correct option (e.g., A, B, C, or D). Then, provide a detailed explanation justifying your choice and explaining why the other options are incorrect.

### Question:
{question}

### Answer:

"""

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0], desc="Đang đánh giá"):
    question_text = row['question']
    ground_truth = str(row['ground_truth']).strip().upper()

    # Định dạng prompt cho mô hình
    prompt = prompt_template.format(question=question_text)

    # Tokenize và gửi lên GPU
    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True).to("cuda")

    # Tạo câu trả lời (dạng giải thích dài)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=400,  # Tăng độ dài để có giải thích chi tiết
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            temperature=0.1,    # Giảm nhiệt độ để câu trả lời nhất quán
            do_sample=True,
        )

    # Decode câu trả lời đầy đủ của model (đây sẽ là cột 'llm_answer')
    full_llm_answer = tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):], skip_special_tokens=True).strip()

    # Trích xuất câu trả lời tóm tắt (A, B, C, D) từ câu trả lời đầy đủ
    # Regex tìm ký tự A-F đầu tiên ở đầu chuỗi (không phân biệt hoa thường)
    match = re.search(r'^\s*([A-F])', full_llm_answer, re.IGNORECASE)
    summary_llmanswer = match.group(1).upper() if match else "N/A" # Lấy ký tự và chuyển thành in hoa, nếu không có thì là N/A

    # Kiểm tra đáp án và tính điểm
    is_correct = 1 if (summary_llmanswer == ground_truth) else 0
    if is_correct == 1:
        correct_answers += 1

    # Lưu kết quả theo đúng định dạng bạn yêu cầu
    results_list.append({
        'index': row['index'],
        'question': question_text,
        'llm_answer': full_llm_answer,
        'summary_llmanswer': summary_llmanswer,
        'ground_truth': ground_truth,
        '_is_correct': is_correct
    })

#5. Tính toán Độ chính xác và Lưu kết quả ---
accuracy = (correct_answers / len(test_df)) * 100 if len(test_df) > 0 else 0

print("\n--- ĐÁNH GIÁ HOÀN TẤT ---")
print(f"Tổng số câu hỏi: {len(test_df)}")
print(f"Số câu trả lời đúng: {correct_answers}")
print(f"Độ chính xác: {accuracy:.2f}%")

# Tạo DataFrame từ list kết quả
results_df = pd.DataFrame(results_list)
RESULTS_CSV_PATH = os.path.join(GDRIVE_PATH, "reasoning_evaluation_results.csv")
# Lưu kết quả chi tiết ra file CSV theo đúng định dạng yêu cầu
results_df.to_csv(RESULTS_CSV_PATH, index=False, encoding='utf-8-sig')
print(f"\nKết quả đánh giá chi tiết đã được lưu tại: {RESULTS_CSV_PATH}")

Đang tải kiến trúc mô hình gốc: meta-llama/Llama-3.2-1B
Đang tải adapter đã fine-tune từ: /content/drive/My Drive/CEH_project/final_checkpoint


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Tải và hợp nhất mô hình thành công.
Đã tải 150 câu hỏi từ /content/drive/MyDrive/CEH_project/data/Bo_B1/B1.csv để đánh giá.


Đang đánh giá: 100%|██████████| 150/150 [25:43<00:00, 10.29s/it]


--- ĐÁNH GIÁ HOÀN TẤT ---
Tổng số câu hỏi: 150
Số câu trả lời đúng: 61
Độ chính xác: 40.67%

Kết quả đánh giá chi tiết đã được lưu tại: /content/drive/My Drive/CEH_project/reasoning_evaluation_results.csv


Test trên B1.csv với mô hình gốc

In [8]:
# Đánh giá mô hình GỐC trên file B1.csv
print("--- Bắt đầu đánh giá mô hình GỐC (chưa fine-tune) trên B1.csv ---")
BASE_MODEL_ID = "meta-llama/Llama-3.2-1B"

bnb_config_test_base = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

print(f"Đang tải mô hình gốc: {BASE_MODEL_ID}")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config_test_base,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

base_model.eval() # Chuyển mô hình sang chế độ đánh giá
print("Tải mô hình gốc thành công.")


# Tải và chuẩn bị Dữ liệu Test (B1.csv) ---
try:
    test_df = pd.read_csv(B1_CSV_PATH)
    test_df.dropna(subset=['question', 'ground_truth'], inplace=True)
    print(f"Đã tải {len(test_df)} câu hỏi từ {B1_CSV_PATH} để đánh giá.")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file B1.csv tại {B1_CSV_PATH}. Hãy chắc chắn bạn đã tải file lên đúng thư mục trên Google Drive.")
    raise


# Vòng lặp Đánh giá ---
correct_answers = 0
results_list = []

# (QUAN TRỌNG) Prompt này phải giống hệt với prompt bạn dùng để đánh giá mô hình đã fine-tune
# để đảm bảo sự so sánh là công bằng.
prompt_template = """### Instruction:
Analyze the following multiple-choice question from a Certified Ethical Hacker (CEH) context. Start your response by stating the single letter of the correct option (e.g., A, B, C, or D). Then, provide a detailed explanation justifying your choice and explaining why the other options are incorrect.

### Question:
{question}

### Answer:

"""

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0], desc="Đang đánh giá mô hình gốc"):
    question_text = row['question']
    ground_truth = str(row['ground_truth']).strip()

    prompt = prompt_template.format(question=question_text)
    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True).to("cuda")

    with torch.no_grad():
        outputs = base_model.generate(
            **inputs,
            max_new_tokens=5,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    generated_text = tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):], skip_special_tokens=True).strip()

    match = re.search(r'^\s*([A-F])', generated_text)
    model_answer = match.group(1) if match else "N/A"

    if model_answer == ground_truth:
        correct_answers += 1

    results_list.append({
        'index': row['index'],
        'question': question_text,
        'ground_truth': ground_truth,
        'model_answer_raw': generated_text,
        'model_answer_parsed': model_answer,
        'is_correct': (model_answer == ground_truth)
    })

# Tính toán và Hiển thị Độ chính xác ---
accuracy = (correct_answers / len(test_df)) * 100 if len(test_df) > 0 else 0

print("\n--- ĐÁNH GIÁ MÔ HÌNH GỐC HOÀN TẤT ---")
print(f"Mô hình được đánh giá: {BASE_MODEL_ID}")
print(f"Tổng số câu hỏi: {len(test_df)}")
print(f"Số câu trả lời đúng: {correct_answers}")
print(f"Độ chính xác (Baseline): {accuracy:.2f}%")

# Lưu kết quả chi tiết ra file CSV ---
results_df = pd.DataFrame(results_list)
results_df.to_csv(RESULTS_CSV_PATH_BASE, index=False, encoding='utf-8-sig')
print(f"\nKết quả đánh giá chi tiết đã được lưu tại: {RESULTS_CSV_PATH_BASE}")

--- Bắt đầu đánh giá mô hình GỐC (chưa fine-tune) trên B1.csv ---
Đang tải mô hình gốc: meta-llama/Llama-3.2-1B
Tải mô hình gốc thành công.
Đã tải 150 câu hỏi từ /content/drive/MyDrive/CEH_project/data/Bo_B1/B1.csv để đánh giá.


Đang đánh giá mô hình gốc: 100%|██████████| 150/150 [00:56<00:00,  2.67it/s]


--- ĐÁNH GIÁ MÔ HÌNH GỐC HOÀN TẤT ---
Mô hình được đánh giá: meta-llama/Llama-3.2-1B
Tổng số câu hỏi: 150
Số câu trả lời đúng: 27
Độ chính xác (Baseline): 18.00%

Kết quả đánh giá chi tiết đã được lưu tại: /content/drive/My Drive/CEH_project/evaluation_results_B1_BASE_MODEL.csv
